# Sorting tables...

In [ ]:
import pandas as pd
parameters_with_filtering    = pd.read_csv('/content/FINAL_TABLE_IC.csv')
parameters_without_filtering = pd.read_csv('/content/ORIGINAL_PARAMETERS_IC.csv')

In [ ]:
parameters_with_filtering.sort_values(by=['e_period', 'chi2']).head()

,CoRoT_ID,period_deleuil,period,e_period,p_deleuil,p,e_p,adivR_deleuil,adivR,e_adivR,b_deleuil,b,e_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei
2078,105793995.0,4.860616,4.84,0.001376,0.1599,0.14,0.001239,9.66,9.64,0.001376,1.04,1.02,0.001583,0.807242,gaussian,NaN,f08,NaN
1106,105793995.0,4.860616,4.84,0.001376,0.1599,0.14,0.001290,9.66,9.64,0.001376,1.04,1.02,0.001499,0.863539,butterworth,n1,f08,NaN
2042,105793995.0,4.860616,4.84,0.001376,0.1599,0.14,0.001290,9.66,9.64,0.001376,1.04,1.02,0.001499,0.876088,gaussian,NaN,f06,NaN
1088,105793995.0,4.860616,4.88,0.001376,0.1599,0.14,0.001290,9.66,9.67,0.001376,1.04,1.03,0.001499,0.883238,butterworth,n1,f07,NaN
1934,105793995.0,4.860616,4.84,0.001407,0.1599,0.14,0.001283,9.66,9.64,0.001407,1.04,1.02,0.001578,0.731365,butterworth,n6,f09,NaN


In [ ]:
# parameters_with_filtering.sort_values(by=['e_p', 'chi2']).head()
parameters_with_filtering[parameters_with_filtering['e_p'] != 0].sort_values(by=['e_p', 'chi2']).head()

,CoRoT_ID,period_deleuil,period,e_period,p_deleuil,p,e_p,adivR_deleuil,adivR,e_adivR,b_deleuil,b,e_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei
171,105819653.0,9.756373,9.74,0.002715,0.0468,0.03,6.661209e-19,13.06,13.04,0.002715,0.66,0.69,0.003279,0.496419,bessel,n2,f01,NaN
333,105819653.0,9.756373,9.74,0.002715,0.0468,0.03,6.661209e-19,13.06,13.04,0.002715,0.66,0.69,0.003279,0.515957,bessel,n3,f01,NaN
495,105819653.0,9.756373,9.74,0.002715,0.0468,0.03,6.661209e-19,13.06,13.04,0.002715,0.66,0.69,0.003279,0.526277,bessel,n4,f01,NaN
2223,105819653.0,9.756373,9.74,0.002715,0.0468,0.03,6.661209e-19,13.06,13.04,0.002715,0.66,0.69,0.003279,0.529004,ideal,NaN,f07,NaN
657,105819653.0,9.756373,9.74,0.002715,0.0468,0.03,6.661209e-19,13.06,13.04,0.002715,0.66,0.69,0.003279,0.532416,bessel,n5,f01,NaN


In [ ]:
parameters_with_filtering.sort_values(by=['e_adivR', 'chi2']).head()

,CoRoT_ID,period_deleuil,period,e_period,p_deleuil,p,e_p,adivR_deleuil,adivR,e_adivR,b_deleuil,b,e_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei
2078,105793995.0,4.860616,4.84,0.001376,0.1599,0.14,0.001239,9.66,9.64,0.001376,1.04,1.02,0.001583,0.807242,gaussian,NaN,f08,NaN
1106,105793995.0,4.860616,4.84,0.001376,0.1599,0.14,0.001290,9.66,9.64,0.001376,1.04,1.02,0.001499,0.863539,butterworth,n1,f08,NaN
2042,105793995.0,4.860616,4.84,0.001376,0.1599,0.14,0.001290,9.66,9.64,0.001376,1.04,1.02,0.001499,0.876088,gaussian,NaN,f06,NaN
1088,105793995.0,4.860616,4.88,0.001376,0.1599,0.14,0.001290,9.66,9.67,0.001376,1.04,1.03,0.001499,0.883238,butterworth,n1,f07,NaN
1934,105793995.0,4.860616,4.84,0.001407,0.1599,0.14,0.001283,9.66,9.64,0.001407,1.04,1.02,0.001578,0.731365,butterworth,n6,f09,NaN


In [ ]:
teste = parameters_with_filtering[parameters_with_filtering['e_b'] != 0].sort_values(by=['e_b', 'chi2']); teste.head()

,CoRoT_ID,period_deleuil,period,e_period,p_deleuil,p,e_p,adivR_deleuil,adivR,e_adivR,b_deleuil,b,e_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei
95,102890318.0,1.508956,1.49,0.00527,0.141,0.13,0.001543,4.62,4.60,0.004364,0.51,0.54,3.425739e-17,57.845964,bessel,n1,f06,NaN
257,102890318.0,1.508956,1.53,0.00527,0.141,0.13,0.001543,4.62,4.61,0.004364,0.51,0.54,3.425739e-17,58.972142,bessel,n2,f06,NaN
77,102890318.0,1.508956,1.49,0.00527,0.141,0.12,0.001543,4.62,4.60,0.004364,0.51,0.54,3.425739e-17,59.125200,bessel,n1,f05,NaN
419,102890318.0,1.508956,1.53,0.00527,0.141,0.13,0.001543,4.62,4.61,0.004364,0.51,0.54,3.425739e-17,59.275835,bessel,n3,f06,NaN
581,102890318.0,1.508956,1.53,0.00527,0.141,0.13,0.001543,4.62,4.61,0.004364,0.51,0.54,3.425739e-17,59.409502,bessel,n4,f06,NaN


# Procedures

1. Ensure that the parameters are compatible: within a range of +- 3 sigma (99.7% confidence)
2. Uncertainty (with filtering) <= Uncertainty (without filtering)
3. Compute percentage reduction:

$$
PR = \frac{\text{uncertantie with filtering} - \text{uncertantie without filtering}}{\text{uncertantie without filtering}} *100
$$

4. Sort from largest to smallest for each parameter for each curve by PR

5. Take the average PR for each curve => Best filtering process


In [ ]:
# @title Analyze results function
import pandas as np
import numpy as np

parameters_with_filtering    = pd.read_csv('/content/FINAL_TABLE_IC.csv')
parameters_without_filtering = pd.read_csv('/content/ORIGINAL_PARAMETERS_IC.csv')

# Delete 102912369 (e_p original = 0)
parameters_with_filtering = parameters_with_filtering.drop(parameters_with_filtering[parameters_with_filtering['CoRoT_ID']==102912369].index)

parameters_without_filtering = parameters_without_filtering.drop(parameters_without_filtering[parameters_without_filtering['CoRoT_ID']==102912369].index)


def analyze_table(ID: int, parameter: str):
  # Selecting lines corresponding to the ID 
  parameters_of_a_specific_id                   = parameters_with_filtering[parameters_with_filtering['CoRoT_ID'] == ID]
  parameters_without_filtering_of_a_specific_id = parameters_without_filtering[parameters_without_filtering['CoRoT_ID'] == ID]

  # How many different uncertainty values do we have?
  # print(parameters_of_a_specific_id['e_'+parameter].value_counts())

  # Keep values in the range of +- 3*sigma
  parameter_referency = parameters_without_filtering_of_a_specific_id[parameter].iloc[0]
  e_parameter_referency = parameters_without_filtering_of_a_specific_id['e_'+parameter].iloc[0]
  valid_parameters_values_tmp = pd.DataFrame(columns=['CoRoT_ID', parameter, parameter+'_deleuil' , 'e_'+parameter, 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei'])
  
  for index in range(len(parameters_of_a_specific_id)):
    actual_parameter = parameters_of_a_specific_id[parameter].iloc[index]
    actual_uncertanties = parameters_of_a_specific_id['e_'+parameter].iloc[index]

    if (actual_parameter > (parameter_referency - 3*e_parameter_referency)) & (actual_parameter < (parameter_referency + 3*e_parameter_referency)):
      valid_parameters_values_tmp = valid_parameters_values_tmp.append([{
        'CoRoT_ID': ID, 
        parameter: actual_parameter,
        parameter+'_deleuil': parameters_of_a_specific_id[parameter+'_deleuil'].iloc[index],
        'e_'+parameter: parameters_of_a_specific_id['e_'+parameter].iloc[index], 
        'chi2': parameters_of_a_specific_id['chi2'].iloc[index], 
        'filter_technique': parameters_of_a_specific_id['filter_technique'].iloc[index], 
        'filter_order': parameters_of_a_specific_id['filter_order'].iloc[index], 
        'filter_cutoff': parameters_of_a_specific_id['filter_cutoff'].iloc[index], 
        'filter_numNei': parameters_of_a_specific_id['filter_numNei'].iloc[index]
    }])
  range_index = np.arange(0, len(valid_parameters_values_tmp))
  valid_parameters_values_tmp = valid_parameters_values_tmp.set_index(range_index)

  # Append parameters and uncertanties without filtering
  valid_parameters_values_tmp['raw_'+parameter] = parameters_without_filtering_of_a_specific_id[parameter].iloc[0]
  valid_parameters_values_tmp['e_raw_'+parameter] = parameters_without_filtering_of_a_specific_id['e_'+parameter].iloc[0]
  valid_parameters_values_tmp = valid_parameters_values_tmp.reindex(['CoRoT_ID', parameter, parameter+'_deleuil', 'e_'+parameter, 'raw_'+parameter, 'e_raw_'+parameter, 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei'], axis=1)


  # Keep parameters that uncertainties were smaller or equal than the original
  count = 0
  for index, row in valid_parameters_values_tmp.iterrows():
    if row['e_'+parameter] > valid_parameters_values_tmp['e_raw_'+parameter].iloc[0]:
      count += 1
      valid_parameters_values_tmp.drop([index], inplace=True)
  # print(f'Deleted {count} rows')

  # Percetage reduction
  pr_column = 'pr_'+parameter

  valid_parameters_values_tmp[pr_column] = 100*(abs(valid_parameters_values_tmp['e_'+parameter] - valid_parameters_values_tmp['e_raw_'+parameter]))/valid_parameters_values_tmp['e_raw_'+parameter]
  valid_parameters_values_tmp = valid_parameters_values_tmp.sort_values(by=pr_column, ascending=False)
  valid_parameters_values_tmp = valid_parameters_values_tmp[['CoRoT_ID', parameter+'_deleuil', parameter, 'e_'+parameter, 'e_raw_'+parameter, 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei', pr_column]]

  return valid_parameters_values_tmp


### Parameter: period

In [ ]:
df_period = pd.DataFrame()

for ID in parameters_without_filtering['CoRoT_ID'].values:
  analysis_results = analyze_table(ID, 'period').iloc[0]
  df_period = df_period.append(analysis_results)
  df_period = df_period[['CoRoT_ID', 'period_deleuil', 'period', 'e_period', 'e_raw_period', 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei', 'pr_period']]
  df_period = df_period.sort_values(by='pr_period', ascending=False)
  df_period.index = np.arange(0, len(df_period))

# df_period.to_csv('table_period.csv')

In [ ]:
df_period[df_period['CoRoT_ID']==105833549]

,CoRoT_ID,period_deleuil,period,e_period,e_raw_period,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei,pr_period
0,105833549.0,2.994279,3.01,0.002209,0.003344,2.913381,butterworth,n1,f06,NaN,33.930635


In [ ]:
# df_period

### Parameter: p

In [ ]:
df_p = pd.DataFrame()

for ID in parameters_without_filtering['CoRoT_ID'].values:
  analysis_results = analyze_table(ID, 'p').iloc[0]
  df_p = df_p.append(analysis_results)
  df_p = df_p[['CoRoT_ID', 'p_deleuil', 'p', 'e_p', 'e_raw_p', 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei', 'pr_p']]
  df_p = df_p.sort_values(by='pr_p', ascending=False)
  df_p.index = np.arange(0, len(df_p))

# df_p.to_csv('table_p.csv')

In [ ]:
df_p[df_p['CoRoT_ID']==101086161]

,CoRoT_ID,p_deleuil,p,e_p,e_raw_p,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei,pr_p
1,101086161.0,0.0892,0.07,0.0,0.000725,8.820598,bessel,n5,f01,NaN,100.0


In [ ]:
# df_p

### Parameter: adivR

In [ ]:
df_adivR = pd.DataFrame()

for ID in parameters_without_filtering['CoRoT_ID'].values:
  analysis_results = analyze_table(ID, 'adivR').iloc[0]
  df_adivR = df_adivR.append(analysis_results)
  df_adivR = df_adivR[['CoRoT_ID', 'adivR_deleuil', 'adivR', 'e_adivR', 'e_raw_adivR', 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei', 'pr_adivR']]
  df_adivR = df_adivR.sort_values(by='pr_adivR', ascending=False)
  df_adivR.index = np.arange(0, len(df_adivR))

# df_adivR.to_csv('table_adivR.csv')

In [ ]:
df_adivR[df_adivR['CoRoT_ID']==101086161]

,CoRoT_ID,adivR_deleuil,adivR,e_adivR,e_raw_adivR,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei,pr_adivR
5,101086161.0,11.93,11.92,0.002094,0.002094,2.855916,butterworth,n2,f09,NaN,0.0


In [ ]:
# df_adivR

### Parameter: b

In [ ]:
############# WARNING #############
## ID: 102890318 
## Parameter: b
## e_b = 0 !!!


df_b = pd.DataFrame()

for ID in parameters_without_filtering['CoRoT_ID'].values:
  analysis_results = analyze_table(ID, 'b').iloc[0]
  df_b = df_b.append(analysis_results)
  df_b = df_b[['CoRoT_ID', 'b_deleuil', 'b', 'e_b', 'e_raw_b', 'chi2', 'filter_technique', 'filter_order', 'filter_cutoff', 'filter_numNei', 'pr_b']]
  df_b = df_b.sort_values(by='pr_b', ascending=False)
  df_b.index = np.arange(0, len(df_b))

# df_b.to_csv('table_b.csv')

In [ ]:
df_b[df_b['CoRoT_ID']==102764809]

,CoRoT_ID,b_deleuil,b,e_b,e_raw_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei,pr_b
0,102764809.0,0.74,0.77,0.0,0.001577,44.814144,bessel,n1,f01,NaN,100.0


In [ ]:
# df_b

# Seeing filtering with details

In [ ]:
# !pip install /content/imt_lightcurve-2.0-py3-none-any.whl --force-reinstall

In [ ]:
import numpy as np
import pandas as pd
from imt_lightcurve.models.lightcurve import LightCurve
from imt_lightcurve.visualization.data_viz import multi_line_plot


# Chosen a LightCurve to simulation process
CURVE_ID = '105833549'

# Importing lightcurve data from github
data = pd.read_csv('https://raw.githubusercontent.com/Guilherme-SSB/IC-CoRoT_Kepler/main/resampled_files/' + CURVE_ID + '.csv')
time = data.DATE.to_numpy()
flux = data.WHITEFLUX.to_numpy()

# Create the LightCurve object
curve = LightCurve(time=time, flux=flux)
# curve.plot()

# Folded curve
folded_curve = curve.fold(CURVE_ID)
# folded_curve.plot()

filtered = curve.butterworth_lowpass_filter(1, 0.6)
filtered.view_filtering_results()

In [ ]:
curve_aux = LightCurve(filtered.time, filtered.filtered_flux)
filtered_folded_curve = curve_aux.fold(CURVE_ID)
multi_line_plot(folded_curve.time, folded_curve.flux, filtered_folded_curve.flux, label_y1='Folded original LC', label_y2='Folded filtered LC', title='Folded original LC x Folded filtered LC')